In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

import wrangle

#### 1. Acquire data from the cloud database.

You will want to end with a single dataframe. Include the logerror field and all other fields related to the properties that are available. You will end up using all the tables in the database.

Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid. - Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction. (Hint: read the docs for the .duplicated method) - Only include properties that have a latitude and longitude value.

In [2]:
zil = wrangle.acquire_zillow()

#### 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [3]:
wrangle.summarize(zil)

DataFrame head: 
        id  basementsqft  bathroomcnt  bedroomcnt  buildingqualitytypeid  \
0  1727539           NaN          3.5         4.0                    NaN   
1  1387261           NaN          1.0         2.0                    NaN   
2    11677           NaN          2.0         3.0                    NaN   
3  2288172           NaN          3.0         4.0                    8.0   
4   781532           NaN          3.0         4.0                    9.0   

   calculatedbathnbr  decktypeid  finishedfloor1squarefeet  \
0                3.5         NaN                       NaN   
1                1.0         NaN                    1465.0   
2                2.0         NaN                       NaN   
3                3.0         NaN                       NaN   
4                3.0         NaN                       NaN   

   calculatedfinishedsquarefeet  finishedsquarefeet12  ...  \
0                        3100.0                3100.0  ...   
1                        1465

In [4]:
nums = [col for col in zil.columns if zil[col].dtype != 'O']
cats = [col for col in zil.columns if col not in nums]

In [5]:
# for col in nums:
#     sns.histplot(data=zil, x=col)
#     plt.title(col+' dist')
#     plt.show()

#### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [6]:
wrangle.nulls_by_col(zil)

,num_rows_missing,perc_rows_missing
id,0,0.000000
basementsqft,52273,99.910168
bathroomcnt,0,0.000000
bedroomcnt,0,0.000000
buildingqualitytypeid,18665,35.674694
calculatedbathnbr,135,0.258028
decktypeid,51931,99.256498
finishedfloor1squarefeet,47949,91.645642
calculatedfinishedsquarefeet,81,0.154817
finishedsquarefeet12,246,0.470183


#### after handeling nulls with the handle_missing_values fx, I would do the following:
- drop buildingqualitytypeid --> don't need a foreign key
- see if there is a mathmatical fx to get to calculated bathnbr, finishedsquarefeet12, fullbathcnt, lotsizesquarefeet, yearbuilt, structuretaxvaluedollarcnt
- use the mean for like home with bath, bed to get calculated finished square feet
- use other homes to determine propertyzoningdesc, regionidcity, regionidzip, heatingorsystemdesc
- look at zoning or type to get unitcnt
- drop the remaining rows with missing values

## PREPARE

#### 1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.

In [7]:
# zil = zil[(zil.unitcnt < 2) | (zil.unitcnt.isnull())]

In [8]:
# zil.unitcnt.value_counts(dropna=False)

In [9]:
# zil = zil[(zil.numberofstories < 3) |(zil.numberofstories.isnull())]

In [10]:
# zil.numberofstories.value_counts(dropna=False)

In [11]:
# zil.basementsqft = zil.basementsqft.fillna(0)

In [12]:
# zil.calculatedbathnbr = zil.calculatedbathnbr.fillna(0)

In [13]:
# zil.typeconstructiondesc = zil.typeconstructiondesc.fillna('None')
# zil.fireplaceflag = zil.fireplaceflag.fillna(0)
# zil.threequarterbathnbr = zil.threequarterbathnbr.fillna(0)
# zil.taxdelinquencyyear = zil.taxdelinquencyyear.fillna(99999)
# zil.taxdelinquencyflag = zil.taxdelinquencyflag.fillna('N')
# zil.calculatedbathnbr = zil.calculatedbathnbr.fillna(0)
# zil.basementsqft = zil.basementsqft.fillna(0)

In [14]:
# zil.taxdelinquencyflag = zil.taxdelinquencyflag.fillna('N')

In [15]:
# zil.taxdelinquencyyear = zil.taxdelinquencyyear.fillna(99999)

In [16]:
# zil.threequarterbathnbr = zil.threequarterbathnbr.fillna(0)

In [17]:
# zil.fireplaceflag = zil.fireplaceflag.fillna(0)

In [18]:
# zil.pooltypeid2 = zil.pooltypeid2.fillna(0)
# zil.pooltypeid10 = zil.pooltypeid10.fillna(0)
# zil.pooltypeid7 = zil.pooltypeid7.fillna(0)

In [19]:
# zil.hashottuborspa = zil.hashottuborspa.fillna(0)

In [20]:
# zil.poolcnt = zil.poolcnt.fillna(0)

In [21]:
# zil.decktypeid = zil.decktypeid.fillna(0)

In [22]:
# zil.fireplacecnt = zil.fireplacecnt.fillna(0)

In [23]:
# zil.fullbathcnt = zil.fullbathcnt.fillna(0)

#### 2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

- The input:
    - A dataframe
    - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
    - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
- The output:
    - The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
- hint:
    - Look up the dropna documentation.
    - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.

In [24]:
# zil = wrangle.handle_missing_values(zil, prop_req_cols=.6, prop_req_rows=.75)
# zil.head()

In [25]:
# zil = zil.drop(columns=['buildingqualitytypeid','propertyzoningdesc','unitcnt','heatingorsystemdesc'])

In [26]:
# zil = zil.dropna()

In [27]:
zil.isnull().sum()

id                                  0
basementsqft                    52273
bathroomcnt                         0
bedroomcnt                          0
buildingqualitytypeid           18665
calculatedbathnbr                 135
decktypeid                      51931
finishedfloor1squarefeet        47949
calculatedfinishedsquarefeet       81
finishedsquarefeet12              246
finishedsquarefeet50            47949
finishedsquarefeet6             52155
fips                                0
fireplacecnt                    45087
fullbathcnt                       135
garagecarcnt                    34336
garagetotalsqft                 34336
hashottuborspa                  50806
latitude                            0
longitude                           0
lotsizesquarefeet                 366
poolcnt                         41243
poolsizesum                     51455
pooltypeid10                    51877
pooltypeid2                     51249
pooltypeid7                     42330
propertycoun

In [28]:
zil.shape

(52320, 58)

#### 3. Encapsulate your work inside of functions in a wrangle_zillow.py module.

In [29]:
# Done!

In [30]:
# def prepare_zillow(df):
#     df = df.drop(columns=['buildingqualitytypeid','propertyzoningdesc','unitcnt','heatingorsystemdesc','id','id.1'])
#     df.fullbathcnt = df.fullbathcnt.fillna(0)
#     df.pooltypeid2 = df.pooltypeid2.fillna(0)
#     df.pooltypeid10 = df.pooltypeid10.fillna(0)
#     df.pooltypeid7 = df.pooltypeid7.fillna(0)
#     df.fireplacecnt = df.fireplacecnt.fillna(0)
#     df.decktypeid = df.decktypeid.fillna(0)
#     df.poolcnt = df.poolcnt.fillna(0)
#     df.hashottuborspa = df.hashottuborspa.fillna(0)
#     df.typeconstructiondesc = df.typeconstructiondesc.fillna('None')
#     df.fireplaceflag = df.fireplaceflag.fillna(0)
#     df.threequarterbathnbr = df.threequarterbathnbr.fillna(0)
#     df.taxdelinquencyyear = df.taxdelinquencyyear.fillna(99999)
#     df.taxdelinquencyflag = df.taxdelinquencyflag.fillna('N')
#     df.calculatedbathnbr = df.calculatedbathnbr.fillna(0)
#     df.basementsqft = df.basementsqft.fillna(0)
#     df.numberofstories.value_counts(dropna=False)
#     wrangle.handle_missing_values(df, prop_req_cols=.6, prop_req_rows=.75)
#     df = df.dropna()
    
#     return df

In [31]:
zil2 = wrangle.prepare_zillow(zil)

In [32]:
zil2.shape

(50596, 39)

In [33]:
zil2.head()

,basementsqft,bathroomcnt,bedroomcnt,calculatedbathnbr,decktypeid,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fireplacecnt,fullbathcnt,...,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,typeconstructiondesc,propertylandusedesc
0,0.0,3.5,4.0,3.5,0.0,3100.0,3100.0,6059.0,0.0,3.0,...,2016.0,537569.0,11013.72,N,99999.0,6.059063e+13,0.025595,2017-01-01,None,Single Family Residential
1,0.0,1.0,2.0,1.0,0.0,1465.0,1465.0,6111.0,1.0,1.0,...,2016.0,376000.0,5672.48,N,99999.0,6.111001e+13,0.055619,2017-01-01,None,Single Family Residential
2,0.0,2.0,3.0,2.0,0.0,1243.0,1243.0,6059.0,0.0,2.0,...,2016.0,479489.0,6488.30,N,99999.0,6.059022e+13,0.005383,2017-01-01,None,Single Family Residential
3,0.0,3.0,4.0,3.0,0.0,2376.0,2376.0,6037.0,0.0,3.0,...,2016.0,36225.0,1777.51,N,99999.0,6.037300e+13,-0.103410,2017-01-01,None,Single Family Residential
4,0.0,3.0,4.0,3.0,0.0,2962.0,2962.0,6037.0,0.0,3.0,...,2016.0,496619.0,9516.26,N,99999.0,6.037461e+13,-0.001011,2017-01-01,None,Single Family Residential


In [34]:
zil2.isnull().sum()

basementsqft                    0
bathroomcnt                     0
bedroomcnt                      0
calculatedbathnbr               0
decktypeid                      0
calculatedfinishedsquarefeet    0
finishedsquarefeet12            0
fips                            0
fireplacecnt                    0
fullbathcnt                     0
hashottuborspa                  0
latitude                        0
longitude                       0
lotsizesquarefeet               0
poolcnt                         0
pooltypeid10                    0
pooltypeid2                     0
pooltypeid7                     0
propertycountylandusecode       0
rawcensustractandblock          0
regionidcity                    0
regionidcounty                  0
regionidzip                     0
roomcnt                         0
threequarterbathnbr             0
yearbuilt                       0
fireplaceflag                   0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
assessmentyear

## MALL CUSTOMERS

In [2]:
import wrangle_mall

#### 1. Acquire data from the customers table in the mall_customers database.

In [7]:
df = wrangle_mall.get_mallcustomer_data()

#### 2. Summarize the data (include distributions and descriptive statistics).

In [8]:
wrangle_mall.summarize(df)

DataFrame head: 
             gender  age  annual_income  spending_score
customer_id                                            
1              Male   19             15              39
2              Male   21             15              81
3            Female   20             16               6
4            Female   23             16              77
5            Female   31             17              40


DataFrame info: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 1 to 200
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   gender          200 non-null    object
 1   age             200 non-null    int64 
 2   annual_income   200 non-null    int64 
 3   spending_score  200 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 7.8+ KB
None


DataFrame describe: 
                count   mean        std   min    25%   50%   75%    max
age             200.0  38.85  13.969007  18.0  28.75  

#### 3. Detect outliers using IQR.

In [10]:
nums = [col for col in df.columns if df[col].dtype != 'O']
cats = [col for col in df.columns if col not in nums]

for col in nums:
    q1,q3 = df[col].quantile([.25,.75])
    iqr = q3-q1
    upper = q3 + 1.5*iqr
    lower = q1 - 1.5*iqr

    df = df[(df[col] > lower) & (df[col] < upper)]

In [13]:
df.shape

(198, 4)

#### 4. Split data into train, validate, and test.

In [15]:
from sklearn.model_selection import train_test_split

In [25]:
train, test = train_test_split(df, test_size = .2, random_state=123)
train, val = train_test_split(train, test_size = .25, random_state=123)

In [27]:
train.shape, val.shape, test.shape

((118, 4), (40, 4), (40, 4))

#### 5. Encode categorical columns using a one hot encoder (pd.get_dummies).

In [20]:
# Map works just as well in this instance

In [19]:
df.gender = df.gender.map({'Female': 0, 'Male': 1})

#### 6. Handles missing values.

In [12]:
df.isnull().sum()

gender            0
age               0
annual_income     0
spending_score    0
dtype: int64

In [21]:
# there are none

#### 7. Scaling

In [22]:
from sklearn.preprocessing import MinMaxScaler

In [28]:
cols_to_scale = df.columns.tolist()

# create copies to not mess with the original dataframes
train_scaled = train.copy()
val_scaled = val.copy()
test_scaled = test.copy()

# create the scaler and fit it
scaler = MinMaxScaler()
scaler.fit(train[cols_to_scale])

# use the scaler to scale the data and resave
train_scaled[cols_to_scale] = pd.DataFrame(scaler.transform(train[cols_to_scale]),
                                           columns = train[cols_to_scale].columns.values).set_index([train.index.values])
val_scaled[cols_to_scale] = pd.DataFrame(scaler.transform(val[cols_to_scale]),
                                           columns = val[cols_to_scale].columns.values).set_index([val.index.values])
test_scaled[cols_to_scale] = pd.DataFrame(scaler.transform(test[cols_to_scale]),
                                           columns = test[cols_to_scale].columns.values).set_index([test.index.values])

In [29]:
train_scaled.head()

,gender,age,annual_income,spending_score
customer_id,,,,
182,0.0,0.269231,0.738739,0.867347
49,0.0,0.211538,0.225225,0.418367
35,0.0,0.596154,0.162162,0.132653
186,1.0,0.230769,0.756757,0.979592
114,1.0,0.019231,0.441441,0.459184


#### 8. Encapsulate your work in a wrangle_mall.py python module.

In [37]:
# Done